![EncabezadoIN](EncabezadoIN.JPG)

## Laboratorio 1 - Agrupación (Clustering)

Integrantes: 
Juan Pablo Lora Hernández - 202012524 
Andrés Francisco Borda Rincón - 
Gabriela Vargas Rojas - 
## Caso 
Los sistemas de recomendación han revolucionado la forma en que los usuarios descubren contenido en plataformas de streaming. Estos sistemas utilizan técnicas de aprendizaje automático para predecir las preferencias de los usuarios y ofrecerles contenido relevante. Así, uno de los desafíos clave para este laboratorio, en la construcción de estos sistemas es identificar las características de las películas que influyen en las calificaciones y gustos de los usuarios.
El comprender mejor el aporte que pueden brindar los sistemas de recomendación a una empresa de streaming, motivó a MovieAlpes a iniciar este proyecto. MovieAlpes es una empresa de streaming que quiere ser más efectiva en sus recomendaciones para mantener a sus usuarios satisfechos y, a su vez, capturar nuevos espectadores. Para ello, ha decidido emprender un proyecto para conocer cuáles son los factores que inciden en las calificaciones positivas de los productos que ofrece y entender mejor a sus usuarios. Para realizar este estudio ha recopilado un conjunto de datos que contiene información detallada sobre un amplio espectro de películas, series y mini series de TV, que incluye género, año de lanzamiento, duración, puntuación, entre otros atributos relevantes.
MovieAlpes los ha contactado para realizar este proceso de caracterización e identificación de patrones que puedan ser utilizados para entender las preferencias de sus usuarios e identificar los atributos que influyen en las calificaciones positivas. La idea es mejorar significativamente la experiencia de estos al proporcionar recomendaciones más relevantes y adaptadas a sus gustos individuales. Dados los procesos internos que en la actualidad lleva a cabo MovieAlpes a nivel de aprendizaje automático, requiere que la solución que brinde al problema plateado, sea basada en una tarea de agrupación.

## 1. Carga de librerías necesarias para implementación

In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots


## 2. Cargar los datos

In [2]:
# Se cargan los datos. 
df_movies=pd.read_csv('./Datos_Agrupación_data_MovieAlpes_data.csv', sep=';', encoding = "ISO-8859-1")

## 3. Entendimiento de los datos 

### 3.1. Perfilamiento de datos

In [3]:
df_movies.shape

(7471, 18)

In [4]:
df_movies.head()

,#,index,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,ordering,region,language,types,attributes,isOriginalTitle,main_genre,secondary_genre
0,1,1,tt0103064,movie,Terminator 2: Judgment Day,0.0,1991.0,137,8.6,1128166.0,17.0,US,en,dvd,\N,0.0,Action,Sci-Fi
1,10,17,tt0231050,tvSeries,ToransufÃÂ´mÃÂ¢: KÃÂ¢robotto,0.0,2000.0,22,6.6,577.0,16.0,US,en,alternative,\N,0.0,Action,Adventure
2,13,23,tt0253474,movie,The Pianist,0.0,2002.0,150,8.5,860569.0,7.0,US,en,imdbDisplay,\N,0.0,Biography,Drama
3,21,55,tt0369339,movie,Collateral,0.0,2004.0,120,7.5,417391.0,6.0,US,en,imdbDisplay,\N,0.0,Action,Crime
4,28,87,tt0439547,movie,2 Filhos de Francisco: A HistÃÂ³ria de ZezÃÂ...,0.0,2005.0,132,6.9,5946.0,5.0,US,en,imdbDisplay,\N,0.0,Biography,Drama


In [5]:
# Revisar las características de los datos: completitud y tipo de dato
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7471 entries, 0 to 7470
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   #                7471 non-null   int64  
 1   index            7471 non-null   int64  
 2   tconst           7471 non-null   object 
 3   titleType        7471 non-null   object 
 4   originalTitle    7471 non-null   object 
 5   isAdult          7470 non-null   float64
 6   startYear        7470 non-null   float64
 7   runtimeMinutes   7470 non-null   object 
 8   averageRating    7470 non-null   float64
 9   numVotes         7470 non-null   float64
 10  ordering         7470 non-null   float64
 11  region           7470 non-null   object 
 12  language         7470 non-null   object 
 13  types            7470 non-null   object 
 14  attributes       7470 non-null   object 
 15  isOriginalTitle  7470 non-null   float64
 16  main_genre       7470 non-null   object 
 17  secondary_genr

In [6]:
# Resumen de los estadisticos de las variables numéricas
df_movies.describe()

,#,index,isAdult,startYear,averageRating,numVotes,ordering,isOriginalTitle
count,7471.000000,7471.000000,7470.0,7470.000000,7470.000000,7.470000e+03,7470.000000,7470.0
mean,4043.482666,5440.458439,0.0,2013.157296,7.475676,7.282830e+04,16.567604,0.0
std,2192.753689,2752.820924,0.0,6.979151,2.771444,1.640233e+05,12.761147,0.0
min,1.000000,1.000000,0.0,1990.000000,6.500000,1.000000e+01,1.000000,0.0
25%,2152.500000,3073.500000,0.0,2008.000000,6.900000,6.265250e+03,6.000000,0.0
50%,4033.000000,5421.000000,0.0,2015.000000,7.300000,1.527900e+04,14.000000,0.0
75%,5914.500000,7641.500000,0.0,2019.000000,7.800000,5.641975e+04,24.000000,0.0
max,7849.000000,10274.000000,0.0,2023.000000,92.000000,2.197234e+06,119.000000,0.0


In [7]:
# Explorar las variables que no son numéricas
name_cols_non_number = df_movies.select_dtypes(include = ['object']).columns
name_cols_non_number

Index(['tconst', 'titleType', 'originalTitle', 'runtimeMinutes', 'region',
       'language', 'types', 'attributes', 'main_genre', 'secondary_genre'],
      dtype='object')

In [8]:
#Revisar el contenido de las variables que no son numéricas, en este caso son categóricas
df_movies[name_cols_non_number].describe()

,tconst,titleType,originalTitle,runtimeMinutes,region,language,types,attributes,main_genre,secondary_genre
count,7471,7471,7471,7470,7470,7470,7470,7470,7470,7470
unique,7466,7,7290,362,4,1,5,1,22,26
top,tt0395843,movie,Home,60,IN,en,imdbDisplay,\N,Drama,Drama
freq,2,4673,3,534,5590,7470,7395,7470,1767,2428
